In [17]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [18]:
def xavier_init(fan_in, fan_out, constant = 1):
    low = -constant * np.sqrt(6.0 / (fan_in + fan_out))
    high = -low
    return tf.random_uniform((fan_in, fan_out), minval = low , maxval = high, dtype=tf.float32)

In [27]:
class AdditiveGaussianNoiseAutoencoder(object):
    def __init__(self, n_input, n_hidden, transfer_function = tf.nn.softplus,
                optimizer = tf.train.AdamOptimizer(), scale=0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.train_scale = scale
        network_weights = self._initialize_weights()
        self.weights = network_weights
        self.x = tf.placeholder(tf.float32,[None, self.n_input])
        self.hidden = self.transfer(tf.add(tf.matmul(self.x+scale*tf.random_normal((n_input,)), 
                                                     self.weights['w1']), self.weights['b1']))
        self.reconstruction = tf.add(tf.matmul(self.hidden, self.weights['w2']), self.weights['b2'])
        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(self.x, self.reconstruction), 2.0))
        self.optimizer = optimizer.minimize(self.cost)
        
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
        
        
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input, self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], dtype=tf.float32))
        all_weights['w2'] = tf.Variable(xavier_init(self.n_hidden, self.n_input))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], dtype=tf.float32))
        return all_weights
    
    
    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer),
                                  feed_dict = {self.x : X, self.scale: self.train_scale})
        return cost
    
    
    def calc_total_cost(self, X):
        return self.sess.run(self.cost, feed_dict={self.x: X, self.scale: self.train_scale})
    
    
    def transform(self, X):
        return self.sess.run(self.hidden, feed_dict={self.x:X, self.scale: self.train_scale})
    
    
    def generate(self, hidden = None):
        if hidden is None:
            hidden = self.random_normal(size = self.all_weights['b1'])
        return self.sess.run(self.reconstruction, feed_dict={self.x:X, self.scale: self.train_scale})
    
    def reconstruct(self, X):
        return self.sess.run(self.reconstruction, feed_dict = {self.x:X, self.scale: self.train_scale})
    
    def getWeigthts(self):
        return self.sess.run(self.weights['w1'])
    
    def getBiases(self):
        return self.sess.run(self.weights['b1'])
    

In [20]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [31]:
def standard_scale(X_train, X_test):
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    return X_train, X_test


def get_random_block_from_data(data, batch_size):
    start_index = np.random.randint(0, len(data) - batch_size)
    return data[start_index:(start_index+batch_size)]

In [22]:
X_trian, X_test = standard_scale(mnist.train.images, mnist.test.images)

In [23]:
n_sample = int(mnist.train.num_examples)
training_epochs = 20
batch_size = 128
display_step = 1

In [28]:
autoencoder = AdditiveGaussianNoiseAutoencoder(n_input=784,
                                              n_hidden = 200,
                                              transfer_function = tf.nn.softplus,
                                              optimizer = tf.train.AdamOptimizer(learning_rate = 1e-3),
                                              scale = 0.1)

In [36]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(n_sample/batch_size)
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_trian, batch_size)
        cost = autoencoder.partial_fit(batch_xs)
        avg_cost+= cost / n_sample * batch_size
        
    if epoch % display_step == 0:
        print("Epoch:",  (epoch+1), "cost=", "{:.9f}".format(avg_cost))

Epoch: 1 cost= 8224.434416477
Epoch: 2 cost= 8357.297662500
Epoch: 3 cost= 7653.939800568
Epoch: 4 cost= 7826.910153409
Epoch: 5 cost= 7371.778486364
Epoch: 6 cost= 8208.237620455
Epoch: 7 cost= 7664.174830114
Epoch: 8 cost= 8744.326719318
Epoch: 9 cost= 7924.734423864
Epoch: 10 cost= 7685.991378409
Epoch: 11 cost= 7725.254612500
Epoch: 12 cost= 7410.010372159
Epoch: 13 cost= 7576.041256818
Epoch: 14 cost= 8042.859627273
Epoch: 15 cost= 7844.836992045
Epoch: 16 cost= 7705.483052273
Epoch: 17 cost= 7781.494654545
Epoch: 18 cost= 7792.001009091
Epoch: 19 cost= 7755.704332955
Epoch: 20 cost= 7830.947928409


In [37]:
print("Total cost " + str(autoencoder.calc_total_cost(X_test)))

Total cost 658874.0
